In [1]:
## Project Eagle Script by Shean Mobed
#Imports
import pandas as pd
import numpy as np
from glob import glob
import os
import re


#MultiFile
multifile = sorted(glob('Multifile/*_Genotyping Result_*.csv'))
multifile

#Concat Files with same cols
df = pd.concat((pd.read_csv(file, comment = '#').assign(filename = file)
          for file in multifile), ignore_index=True)
#Date Extraction
File1 = df.filename.get(0)
with open (File1) as GenotypingResults:
    for line in GenotypingResults:
        if line[0:13] == '# Exported On':
            date = line[15:41].replace(':','_').replace('\n','')
                    
date2 = date[:10]

date3 = date[:17].replace('-','').replace(' ','').replace('_','')

date4 = date[:19].replace('_',':') + ' UTC'
#File name
basename = os.path.splitext(os.path.basename(File1))[0]
eagle = basename.split('_')[0]
print(eagle)
print(date)
print(date2)
print(date3)
print(date4)

GLS-GP-006334
2021-09-27 06_10_40 PM BST
2021-09-27
202109270610
2021-09-27 06:10:40 UTC


In [2]:
##Simplify Table
df = df[['Sample','Call','SNP Assay','Confidence']]
df['Sample'] = df['Sample'].astype('str')
# Remove 
sample = df['Sample'].str.len() == 11
df = df.loc[sample]
#Sample0
#sample0 = df[(df['Sample'] == '0') | 
             #(df['Sample'] == 'PosCon') | 
             #(df['Sample'] == 'NTC') | 
             #(df['Sample'] == 'PHE') |
             #(df['Sample'] == 'Empty')
            #]

#df = df.drop(sample0.index)


#Blank Sample
df['Sample'].replace('',np.nan, inplace = True)
df.dropna(subset = ['Sample'], inplace = True)

#Rename Cols
df.rename(columns={'SNP Assay': 'CH1-Target',
         'Call': 'CH1-Result',
          'Confidence': 'CH1-Conf'},
         inplace=True)
#Unknown
unknown = df.loc[df.Sample.str.contains("Unknown",case=False)]
df = df.drop(unknown.index)

In [3]:
## Duplicates Troubleshooter

#Duplicate Barcodes error Raise
bcdups = df.groupby(['Sample']).size() > 4
bcdups = pd.DataFrame(bcdups)
bcdups = bcdups.reset_index()
bcdups = bcdups.rename(columns = { 0:'Bool'})
bcdups = bcdups[(bcdups['Bool']==True)]
bcdups = bcdups.loc[bcdups.Bool == True]

#print(bcdups.Sample.count())
bcdups.to_csv('dups.csv')

try:
    if bcdups['Sample'].count() > 0 :
        raise Exception
except Exception:
    print("Check for duplicate barcodes in files!, duplicates can be found in dups.csv")
else: print('No duplicate barcodes :)')

#SNP assay column duplicates error Raise

snpdups = df.groupby(['CH1-Target']).size()
snpdups = pd.DataFrame(snpdups)
snpdups = snpdups.reset_index()
snpdups = snpdups.rename(columns = { 0:'Count'})
samplecount = df.index.size / 4

try:
    if snpdups['Count'][0] > samplecount :
        raise Exception
except Exception:
    print("Check for error in SNP assay column in the files!")
else: print('No error found in SNP assay column')

No duplicate barcodes :)
Check for error in SNP assay column in the files!


In [4]:
##Mutation Caller
#Duplicate CH1-Result
df['CH1-Result-Copy'] = df['CH1-Result']

# Fancy Split
dfsplit = df['CH1-Result'].str.rsplit('/', n=1)
dfsplit2 = dfsplit.str.get(0)
dfsplit3 = dfsplit2.replace('.*_wt.*$', 'wt', regex=True).replace('.*_mt.*$','mt', regex=True).replace('.*_mut.*$','mt', regex=True)
df['CH1-Result'] = dfsplit3

# Fancy Split 2
dfsplit = df['CH1-Result-Copy'].str.rsplit('/', n=1)
df['CH1-Result-1'] = dfsplit.str.get(0)
df['CH1-Result-2'] = dfsplit.str.get(1)
RS1 = df['CH1-Result-1'].str.split('_')
RS2 = df['CH1-Result-2'].str.split('_')
df['CH1-Result-1'] = RS1.str.get(1)
df['CH1-Result-2'] = RS2.str.get(1)
df['CH1-Split'] = df['CH1-Result-1'] + '_' + df['CH1-Result-2']


## Add Cols Date/ Lab ID/ 

#Date
df['Date Tested']= date4
#lab ID
df['Lab ID']='GLS'
#testKit
df['testKit']='TaqMan-GT2'

#Result Tree
Rconds = [
    df['CH1-Split'] == 'wt_wt',
    (df['CH1-Split'] == 'mt_mt') | (df['CH1-Split'] == 'mut_mut'),
    (df['CH1-Split'] == 'wt_mt') | (df['CH1-Split'] == 'mt_wt') | 
    (df['CH1-Split'] == 'wt_mut') | (df['CH1-Split'] == 'mut_wt') ,
    df['CH1-Result'] == 'Undetermined',
    df['CH1-Result'] == 'No Amplification',
]

Rchoices =[
    'No Mutation',
    'Mutation',
    'n/a',# previously 'het'
    'n/a',
    'n/a',
]

df['Result'] = np.select(Rconds,Rchoices)

#Correct Header Layout
dffinal = df[['Sample', 'Result', 'Date Tested', 'Lab ID', 'testKit', 'CH1-Target', 'CH1-Result', 'CH1-Conf']]
dffinal = dffinal.sort_values(by = ['Sample','CH1-Target']).set_index('Sample').reset_index()
df.columns = df.columns.str.replace("'"," ")
#automatic file name

eaglefile = '(' + eagle + ')'

labid = 'GLS'

#datetested = df['Date Tested'].get(0)

spacer = "_"

endcard1 = '-mut-results.csv'

#output to Individual reports
outfilename1 = eaglefile + spacer + labid + spacer + date + endcard1
outfilepath1 = 'reports/Individual Eagle Reports/'
dffinal.to_csv(outfilepath1 + outfilename1, index=False)

#output for Jodie
outfilename2 = labid + date3 + endcard1
outfilepath2 = 'reports/'
dffinal.to_csv(outfilepath2 + outfilename2, index=False)

#Head
#dffinal.tail()
#mutsize = dffinal.groupby(['CH1-Target','CH1-Result']).size()
#mutsize

#df.dtypes

In [5]:
## VARiants Caller
vc = df.pivot_table(index= ['Sample'], columns= 'CH1-Target', values= 'Result', aggfunc=lambda x: ' '.join(x))
vc.replace(np.nan, 'n/a', inplace = True)

#Lineage Tree
wt = 'No Mutation'
mt = 'Mutation'
het = 'Het'
NA = 'n/a'
Lconds = [
    (vc['P681R'] == wt) & (vc['E484K'] == wt) & (vc['K417N'] == wt) & (vc['K417T'] == wt),
    (vc['P681R'] == wt) & (vc['E484K'] == wt) & (vc['K417N'] == wt) & (vc['K417T'] == wt),
    (vc['P681R'] == wt) & (vc['E484K'] == mt) & (vc['K417N'] == mt) & (vc['K417T'] != wt),
    (vc['P681R'] == wt) & (vc['E484K'] == mt) & (vc['K417N'] != wt) & (vc['K417T'] == mt),
    (vc['P681R'] == NA) | (vc['E484K'] == NA) | (vc['K417N'] == NA) | (vc['K417T'] == NA),
    (vc['P681R'] == het) | (vc['E484K'] == het) | (vc['K417N'] == het) | (vc['K417T'] == het),
    (vc['P681R'] == wt) & (vc['E484K'] == mt) & (vc['K417N'] == wt) & (vc['K417T'] == wt),
    (vc['P681R'] == mt) #& (vc['E484K'] != mt) & (vc['K417N'] != mt) & (vc['K417T'] != mt) 
    
  
] 

Lchoices = [
    'B.1',
    'B.1.1.7',
    'B.1.351',
    'P.1',
    'NA',# previously het
    'NA',
    'A.23.1',
    'B.1.617.2'
   
]

vc['Result'] = np.select(Lconds,Lchoices, default = 'Undetermined')

#Format Cols
#Date
vc['Date Tested'] = date
#lab ID
vc['Lab ID'] = 'GLS'
#testKit
vc['testKit'] = df['testKit']
#Lineage + Result
vc['Result Type'] = 'Lineage'
# Second Output Layout
vardf = vc[['Result Type', 'Result', 'Date Tested', 'Lab ID']]

#automatic file name 2

labid = vc['Lab ID'].get(0)

datetested = vc['Date Tested'].get(0)

spacer = "_"

endcard2 = 'lineage_report.csv'

# Individual report
outfilename3 = eaglefile + spacer + labid + spacer + date + spacer + endcard2
vardf.to_csv(outfilepath1 + outfilename3, index=True,)

#output for Jodie
outfilename4 = labid + spacer + date + spacer + endcard2
vardf.to_csv(outfilepath2 + outfilename4, index=True)

#Head Vardf
vardf.head()
linsize = vardf.groupby(['Result']).size()
linsize

Result
B.1.617.2    417
NA            38
dtype: int64

In [6]:
##Mutation Count File

#Mutation Count DF
mcdf = df[['CH1-Target', 'CH1-Result']]
#Drop No Amp
namp = mcdf[(mcdf['CH1-Result'] == 'No Amplification')| (mcdf['CH1-Result'] == 'Undetermined')]
mcdf = df.drop(namp.index)
#Count Dataframe
mcdf = mcdf.groupby(['CH1-Target','CH1-Result']).size().reset_index(name = eagle)
mcdf = pd.pivot_table(mcdf,
                       index=['CH1-Target','CH1-Result'],
                       values= eagle,                            
                       fill_value = 0,
                       dropna=False)

#automatic file name 3

endcard3 = 'mutation_count.csv'

outfilename5 = labid + spacer + eagle + spacer + endcard3
outfilepath3 = 'counts/'
mcdf.to_csv(outfilepath3 + outfilename5, index=True)

In [7]:
##Mutations Summary Report
counts = sorted(glob('counts/*mutation_count*.csv'))
#Summary Table
msdf = pd.concat((pd.read_csv(f, index_col = ['CH1-Result', 'CH1-Target']) 
                for f in counts), axis =1)
msdf = msdf.rename(columns = str.title).sort_index(axis = 1)

#Total sum per row: 
msdf['Total'] = msdf.sum(axis=1)
total = 'Total'
totalcol = msdf.pop(total)
msdf.insert(0,total,totalcol)

#automatic file name 4

endcard4 = 'eagle_mutation_report_' + date + '.csv'

outfilename6 = labid + spacer + endcard4

msdf.to_csv(outfilepath3 + outfilename6, index=True)

In [8]:
## Tally Lineage
linagg = vc.groupby(['Result'], as_index=True).size().reset_index(name = eagle)
namp = linagg[(linagg['Result'] == 'NA')| (linagg['Result'] == 'Undetermined')]
linagg = linagg.drop(namp.index)

#automatic file name 5

endcard5 = 'lineage_count.csv'

outfilename7 = labid + spacer + eagle + spacer + endcard5
outfilepath4 = 'lineage/'
linagg.to_csv(outfilepath4 + outfilename7, index=False)

In [9]:
## Lineage Summary Report
#Summary Table
lins = sorted(glob('lineage/*lineage_count*.csv'))
lsdf = pd.concat((pd.read_csv(f, index_col = ['Result']) 
                for f in lins), axis =1)

lsdf = lsdf.rename(columns = str.title).sort_index(axis = 1)
#lsdf = lsdf.rename(columns={'Result': 'Lineage'},inplace=True)

#Total sum per row: 
lsdf['Total'] = lsdf.sum(axis=1)
total = 'Total'
totalcol = lsdf.pop(total)
lsdf.insert(0,total,totalcol)
lsdf = lsdf.fillna(0).astype(int)

#automatic file name 6

endcard6 = 'eagle_lineage_report_' + date + '.csv'

outfilename8 = labid + spacer + endcard6

lsdf.to_csv(outfilepath4 + outfilename8, index=True)

In [10]:
## Tally Combos
cdf = df[['Sample','CH1-Target','CH1-Split']]
cdf = cdf.replace(np.nan,'N/A')
cdf = cdf.reset_index(drop=True)

#Changing mut to mt
cdf['CH1-Split'] = cdf['CH1-Split'].replace('mut_mut','mt_mt')

#Pivot Table
cdf = cdf.pivot(index = ['Sample'], columns = 'CH1-Target')

#Sample Count Row
samplecount = cdf.index.size
samplecountdf = pd.DataFrame({'Combo': 'Total', date2 : samplecount}, index = [0])

#Dropping MultiIndex
cdf.columns = ['_'.join(col) for col in cdf.columns.values]
cdf.rename(columns={'CH1-Split_E484K': 'E484K',
                   'CH1-Split_K417N': 'K417N',
                   'CH1-Split_K417T': 'K417T',
                   'CH1-Split_P681R': 'P681R'},
         inplace=True)

#cdf = cdf.reset_index(drop=True)

# Removing Wildtypes and N/a / Defining Heterogenous Mutants
het = ('wt_mut' , 'mut_wt' , 'wt_mt' , 'mt_wt')
cdf = cdf.replace(het,'').replace('wt_wt', '').replace('N/A', '')
cdf = cdf.dropna()

# Substituting mutant calls for their names

cdf.loc[(cdf.E484K == 'mt_mt'),['E484K']] = 'E484K'
cdf.loc[(cdf.K417T == 'mt_mt'),['K417T']] = 'K417T'
cdf.loc[(cdf.K417N == 'mt_mt'),['K417N']] = 'K417N'
cdf.loc[(cdf.P681R == 'mt_mt'),['P681R']] = 'P681R'


# Naming Het Combos
#cdf.loc[(cdf.N501Y == 'Het'),['N501Y']] = 'N501Y (Het)'
#cdf.loc[(cdf.E484K == 'Het'),['E484K']] = 'E484K (Het)'
#cdf.loc[(cdf.K417T == 'Het'),['K417T']] = 'K417T (Het)'
#cdf.loc[(cdf.K417N == 'Het'),['K417N']] = 'K417N (Het)'

# Joining mutant call columns into one result column
cdf['Combo'] = cdf[['E484K', 'K417N','K417T','P681R']].agg('+'.join, axis=1)
#Stripping excess '+' separators
cdf['Combo'] = cdf['Combo'].str.strip('+')
cdf['Combo'] = cdf['Combo'].str.replace('\++', '+', regex=True)
cdf['Combo'] = cdf['Combo'].replace('',np.nan)

# Counting mutation combos
combo = cdf.groupby(['Combo']).size().reset_index(name = date2)
combo = pd.concat([samplecountdf, combo[:]]).reset_index(drop = True)

#automatic file name 
labid ='GLS'
spacer = "_"
endcard7 = 'combo.csv'
outfilename9 = labid + spacer + date2 + spacer + endcard7
outfilepath5 = 'Combo/'
combo.to_csv(outfilepath5 + outfilename9, index=False)

In [11]:
## Combo Summary Report
#Summary Table
combos = sorted(glob('Combo/*combo*.csv'))
csdf = pd.concat((pd.read_csv(f, index_col = ['Combo']) 
                for f in combos), axis = 1)

#Total sum per row: 
csdf['Total'] = csdf.sum(axis=1)
total = 'Total'
totalcol = csdf.pop(total)
csdf.insert(0,total,totalcol)

#Custom Index
ComboIndex = ['Total','N501Y','E484K','E484K+N501Y',
              'K417N','E484K+K417N','K417N+N501Y','E484K+K417N+N501Y',
              'K417T','E484K+K417T+N501Y','P681R']
csdf = csdf.reindex(ComboIndex)
csdf = csdf.fillna(0).astype(int)

#automatic file name 2

endcard8 = 'combination_report_' + date2 + '.csv'

outfilename10 = labid + spacer + endcard8

csdf.to_csv(outfilepath5 + outfilename10, index=True)